# Example 5: Stickleback morphometrics - body and armor-plate shape

Variation in continuous phenotypic traits like shape or area of certain structures are difficult to quantify with landmarks, because they are too complex or have no underlying assumption of homology. In this example, the number and area of armor plating was measured as a continuous trait in a two-step process: first, a mask was set around the posterior region that contains the plates, second, the red channel (highest signal-to-noise-ratio) of the image was thresholded. 

In this example we use the `watershed` algorithm, which helps to separate detected objects into "peaks of wanted information" and "valleys of unwanted information". The principle is explained here: https://docs.opencv.org/master/d3/db4/tutorial_py_watershed.html 

<div class="row; text-align: left">
    
<div class="col-md-6">
    
![Before](_assets/ex5_before.jpg)
    
**Input** - Stained threespine stickleback. The size of plates at a given plate index varies within at between ecotypes (e.g. lake and stream morphs)  
</div>
<div class="col-md-6">

![After](_assets/ex5_after.jpg)
    
**Results** - After applying adaptive thresholding and watershed algorithms, the plates separate. Where this is not the case they can be separated manually.
</div>
</div>

## High throughput workflow

First we need to create a project, as described in [Tutorial 3](tutorial_3_managing_projects_1.ipynb) and [Tutorial 4](tutorial_4_managing_projects_2.ipynb). 

**Note:** If you have already created a project you can skip the following steps and load the project with `pp.project.load("path/to/project")`.

In [1]:
import phenopype as pp

## relative from phenopype-master/tutorials
project_root = r"../_temp/output/ex5_project"
image_dir = "images"
reference_image = "images/stickleback_side.jpg"

### Create project

In [4]:
ex5 = pp.project(root_dir=project_root)

--------------------------------------------
Phenopype will create a new project at
E:\git_repos\phenopype\_temp\output\ex5_project

Proceed? (y/n)
y

project attributes written to E:\git_repos\phenopype\_temp\output\ex5_project\attributes.yaml
--------------------------------------------


This will create a folder structure and allows for easy creation of the `pype`-configuration files needed for high throughput. First we add the image files in the directory, but only "stickle1", "stickle2", and "stickle3". 

In [6]:
ex5.add_files(image_dir=image_dir, include="stickle", exclude=["side","top"])

--------------------------------------------
phenopype will search for files at

E:\git_repos\phenopype\tutorials\images

using the following settings:

filetypes: ['jpg', 'JPG', 'jpeg', 'JPEG', 'tif', 'png'], include: stickle, exclude: ['side', 'top'], raw_mode: copy, search_mode: dir, unique_mode: path

Found image stickle1.JPG - phenopype-project folder 0__stickle1 created
dirpath defaulted to file directory - E:\git_repos\phenopype\tutorials\images
Directory to save files set at - E:\git_repos\phenopype\tutorials\images
no meta-data found
Found image stickle2.JPG - phenopype-project folder 0__stickle2 created
dirpath defaulted to file directory - E:\git_repos\phenopype\tutorials\images
Directory to save files set at - E:\git_repos\phenopype\tutorials\images
no meta-data found
Found image stickle3.JPG - phenopype-project folder 0__stickle3 created
dirpath defaulted to file directory - E:\git_repos\phenopype\tutorials\images
Directory to save files set at - E:\git_repos\phenopype\tut

Now we add the appropriate configuration file. As for the other examples I have created a preset ("ex5") with appropriate settings, which is passed to the pype using `config_preset="ex5"`.

In [7]:
ex5.add_config(name = "v1", config_preset="ex5")
print(pp.presets.ex5)

pype config generated from ex5.
pype_v1.yaml created for 0__stickle1
pype_v1.yaml created for 0__stickle2
pype_v1.yaml created for 0__stickle3

preprocessing:
- create_mask:
    tool: polygon
- find_scale
- enter_data:
    columns: ID
segmentation:
- blur:
    kernel_size: 9
- threshold:
    method: adaptive
    blocksize: 99
    constant: 3
    channel: red
- morphology:
    operation: close
    shape: ellipse
    kernel_size: 3
    iterations: 3
- watershed:
    distance_cutoff: 0.8
# - draw:
    # line_colour: white
    # overwrite: True
- find_contours:
    retrieval: ccomp
    min_area: 100
measurement:
visualization:
- select_canvas:
    canvas: red
- draw_contours:
    line_width: 2
    label_width: 0
    label_size: 1
    fill: 0.3
- draw_masks
export:
- save_contours:
    overwrite: true



Now we add reference image to create a scale-template so we can adjust our landmark coordinate space. This is important if for example the distance between the camera and your sample changes.  

<center>
<div style="width:500px; text-align: left">
    
![Adding a scale](_assets/ex2_scale.gif)
    
</div>
</center>

In [9]:
ex5.add_scale(reference_image = reference_image, overwrite=True)

- scale template saved under E:\git_repos\phenopype\_temp\output\ex5_project\scale_template.jpg.
- measure pixel-to-mm-ratio
Scale set
- add column length
Template selected
added scale information to 0__stickle1
added scale information to 0__stickle2
added scale information to 0__stickle3


Afterwards, we save the project (to the root folder). 

In [10]:
pp.project.save(ex5)

Project data saved under E:\git_repos\phenopype\_temp\output\ex5_project\project.data.


### Load project

In [11]:
ex5 = pp.project.load(project_root)

--------------------------------------------
Project loaded from 
E:\git_repos\phenopype\_temp\output\ex5_project
--------------------------------------------


### The template
The basic points of the preprocessing procedure are:

1. Draw a mask around the area of interest (i.e. the plates)
2. Let the algorithm find the scale
3. Enter the ID from the reference card into the mask

<center>
<div style="width:500px; text-align: left" >
    
![Create masks](_assets/masks2.gif)
    
</div>
</center>

The rest is automatic: a watershed algorithm helps to separate the plates. Sensitivity of the algorithm can be mostly controlled with the `distance_cutoff` argument, but also playing around with the thresholding arguments (`blocksize` and `constant`) can help to improve results. 

Sometimes the overall results are goog except for one or two cases of plates that touch each other. In this case, the `draw` function can be used to separate those plates without changing the whole procedure. 

Now we can run the `pype` routine with a simple loop on `ex5.dirpaths`, which is a list of all project directories that contain the copied raw images and the config files we generated before. Interrupt the loop with Esc. To resume to the point where you left, add the `skip` argument so directory with processed files are not run again.

In [12]:
for dirpath in ex5.dirpaths:
    out = pp.pype(dirpath, name="v1", skip=True) 

E:\git_repos\phenopype\_temp\output\ex5_project\data\0__stickle1\pype_config_v1.yaml


------------+++ new pype iteration 2020:05:18 19:34:15 +++--------------


AUTOLOAD
- template scale information loaded from attributes.yaml
- template loaded from root directory
PREPROCESSING
create_mask
- create mask
find_scale
---------------------------------------------------
Reference card found with 241 keypoint matches:
template image has 36 pixel per mm.
current image has 34.8 pixel per mm.
= 96.782 % of template image.
---------------------------------------------------
enter_data
- add column ID
SEGMENTATION
blur
threshold
- include mask "mask1" pixels
- exclude mask "scale" pixels
morphology
watershed
find_contours
VISUALIZATION
select_canvas
- red channel
draw_contours
draw_masks
 - show mask: mask1.
 - show mask: scale.
EXPORT
save_contours
- contours saved under E:\git_repos\phenopype\_temp\output\ex5_project\data\0__stickle1\contours_v1.csv.
AUTOSAVE
save_canvas
- canvas saved under E

SystemExit: 

TERMINATE (by user)

In [13]:
## inspect results
## DataFrame
out.container.df_contours.drop(columns=["order", "idx_child", "idx_parent", "coords"])

,filename,width,height,size_ratio_original,template_px_mm_ratio,current_px_mm_ratio,ID,contour,center,diameter,area
0,stickle1.JPG,2400,1600,1,36,34.8,142501,1,"(1770, 400)",14,122
1,stickle1.JPG,2400,1600,1,36,34.8,142501,2,"(1800, 399)",20,207
2,stickle1.JPG,2400,1600,1,36,34.8,142501,3,"(1743, 396)",20,193
3,stickle1.JPG,2400,1600,1,36,34.8,142501,4,"(1717, 396)",24,232
4,stickle1.JPG,2400,1600,1,36,34.8,142501,5,"(1690, 391)",39,426
...,...,...,...,...,...,...,...,...,...,...,...
10,stickle1.JPG,2400,1600,1,36,34.8,142501,11,"(1521, 365)",101,1235
11,stickle1.JPG,2400,1600,1,36,34.8,142501,12,"(1485, 360)",114,1559
12,stickle1.JPG,2400,1600,1,36,34.8,142501,13,"(1453, 355)",119,1638
13,stickle1.JPG,2400,1600,1,36,34.8,142501,14,"(1423, 355)",128,1755


In [14]:
## image
pp.show_image(out.container.canvas)